<a href="https://colab.research.google.com/github/saisrikanthmadugula/rkAMM-Simulation-Framework/blob/main/Integrated_MAS_rkAMM_Simulation_Framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# --- MODEL CONSTANTS (for Dynamic MAS Simulation) ---
REPAY_BUMP = 3        # Reputation increase for repayment
DEFAULT_PENALTY = -15 # Reputation decrease for default
MAX_COLLATERAL = 0.8  # Max required collateral is 80% (for 0-rep borrower)
MIN_P_D = 0.01        # 1% p_d (for 100-rep borrower)
MAX_P_D = 0.80        # 80% p_d (for 0-rep borrower)

# ==============================================================================
# --- AGENT DEFINITIONS (The Core Framework) ---
# ==============================================================================

class RiskAssessmentAgent:
    """
    AGENT 1: The STATEFUL AI Agent (from V-11 Paper Logic)

    This agent is stateful. It maintains a memory of each
    borrower's reputation and updates it based on loan outcomes.
    It dynamically determines the required collateral and p_d.
    """
    def __init__(self, initial_borrowers):
        # Memory of all borrowers and their reputation scores
        # Initialize all borrowers with a "newer" profile (per V-11 C.2)
        self.borrower_reputations = {
            borrower_id: np.clip(np.random.normal(loc=45, scale=10), 10, 80)
            for borrower_id in initial_borrowers
        }

    def get_reputation(self, borrower_id):
        return self.borrower_reputations.get(borrower_id)

    def assess_loan_terms(self, borrower_id):
        """
        Generates p_d and C based on the borrower's current reputation.
        """
        reputation = self.get_reputation(borrower_id)

        # 1. Calculate p_d (Probability of Default)
        # Linear model: 100-rep -> MIN_P_D, 0-rep -> MAX_P_D
        p_d = MAX_P_D - (reputation / 100) * (MAX_P_D - MIN_P_D)

        # 2. Calculate C (Required Collateral)
        # Linear model: 100-rep -> 0% C, 0-rep -> MAX_COLLATERAL
        collateral_level = MAX_COLLATERAL * (1.0 - (reputation / 100))

        return np.clip(p_d, MIN_P_D, MAX_P_D), np.clip(collateral_level, 0, MAX_COLLATERAL)

    def update_reputation(self, borrower_id, outcome):
        """
        The feedback loop. Updates reputation based on loan outcome.
        """
        if outcome == "Repaid":
            self.borrower_reputations[borrower_id] += REPAY_BUMP
        else: # Default
            self.borrower_reputations[borrower_id] += DEFAULT_PENALTY

        # Clip reputation to be between 0 and 100
        self.borrower_reputations[borrower_id] = np.clip(
            self.borrower_reputations[borrower_id], 0, 100
        )

class PricingAgent:
    """
    AGENT 2: The rkAMM Pricing Engine (from Esteva et al. 2023)

    Uses the Reverse Kelly AMM logic from Esteva et al. (2023)
    to calculate the required premium for a loan.
    """
    def __init__(self):
        pass

    def calculate_premium(self, p_d, collateral_level, loan_amount=1.0):
        """
        Calculates the premium (P) based on rkAMM logic.
        """
        L = loan_amount * (1.0 - collateral_level)

        if L <= 0 or p_d <= 0:
            return 0.0, L
        if p_d >= 1.0:
            return np.inf, L

        # The core rkAMM formula (Esteva et al., 2023)
        premium = (L * p_d) / (1.0 - p_d)

        return premium, L

class BlockchainEnforcementAgent:
    """
    AGENT 3: The Blockchain Simulator

    Simulates the execution of the loan and its outcome.
    """
    def __init__(self):
        pass

    def execute_loan(self, p_d):
        """
        Simulates if a loan defaults or is repaid based on its p_d.
        """
        return "Repaid" if np.random.rand() >= p_d else "Default"

# ==============================================================================
# --- SIMULATION 1: Dynamic MAS (Validates V-11 Appendix) ---
# ==============================================================================

def run_dynamic_mas_simulation(n_borrowers=500, loans_per_borrower=20):
    """
    Runs the dynamic simulation with a fixed pool of borrowers
    taking out multiple loans to test the "virtuous cycle."
    """
    print(f"--- Running Dynamic MAS Simulation (V-11 Validation) ---")
    print(f"Model: {n_borrowers} borrowers, {loans_per_borrower} loans each.")

    borrower_ids = list(range(n_borrowers))

    # Initialize all three agents
    risk_agent = RiskAssessmentAgent(borrower_ids)
    pricing_agent = PricingAgent()
    blockchain_agent = BlockchainEnforcementAgent()

    results = []
    total_loan_id = 0
    total_loans = n_borrowers * loans_per_borrower

    for loan_num in range(loans_per_borrower):
        np.random.shuffle(borrower_ids)

        for borrower_id in borrower_ids:
            # 1. Assess risk & terms (from AI Agent)
            current_rep = risk_agent.get_reputation(borrower_id)
            p_d, collateral_level = risk_agent.assess_loan_terms(borrower_id)

            # 2. Price the loan (from rkAMM Agent)
            premium, loss_given_default = pricing_agent.calculate_premium(
                p_d, collateral_level
            )

            # 3. Execute loan (from Blockchain Agent)
            outcome = blockchain_agent.execute_loan(p_d)

            # 4. Feedback loop: Update reputation
            risk_agent.update_reputation(borrower_id, outcome)

            # 5. Store results
            results.append({
                "total_loan_id": total_loan_id,
                "loan_round": loan_num,
                "borrower_id": borrower_id,
                "initial_reputation": current_rep,
                "p_d": p_d,
                "collateral_level": collateral_level,
                "premium": premium,
                "outcome": outcome,
                "final_reputation": risk_agent.get_reputation(borrower_id)
            })
            total_loan_id += 1

        # Print progress
        sys.stdout.write(f"\r  Processing loan round {loan_num + 1}/{loans_per_borrower}...")
        sys.stdout.flush()

    print("\nDynamic simulation complete.")
    return pd.DataFrame(results)

def plot_dynamic_results(df_results):
    """
    Generates the two key validation plots for the V-11 Appendix.
    """
    print("Generating validation plots...")

    # 1. Plot 1: Reputation Score Distribution (Validates V-11 C.2)
    final_reputations = df_results[
        df_results['loan_round'] == df_results['loan_round'].max()
    ]['final_reputation']

    plt.figure(figsize=(14, 6))
    plt.subplot(1, 2, 1)
    sns.histplot(final_reputations, bins=20, kde=True)
    plt.title('Final Reputation Score Distribution (V-11 C.2)', fontsize=14)
    plt.xlabel('Reputation Score', fontsize=12)
    plt.ylabel('Number of Borrowers', fontsize=12)

    # 2. Plot 2: Collateral Reduction (Validates V-11 C.1)
    df_repaid = df_results[df_results['outcome'] == 'Repaid'].copy()

    # Calculate the change in collateral
    df_repaid['next_collateral'] = MAX_COLLATERAL * (1.0 - (df_repaid['final_reputation'] / 100))
    df_repaid['collateral_change'] = df_repaid['next_collateral'] - df_repaid['collateral_level']
    avg_collateral_change = df_repaid['collateral_change'].mean()

    avg_collateral_per_round = df_results.groupby('loan_round')['collateral_level'].mean()

    plt.subplot(1, 2, 2)
    avg_collateral_per_round.plot(marker='o')
    plt.title('Avg. Required Collateral Over Time (V-11 C.1)', fontsize=14)
    plt.xlabel('Loan Round', fontsize=12)
    plt.ylabel('Average Collateral Level', fontsize=12)
    plt.gca().yaxis.set_major_formatter(plt.FuncFormatter('{:.1%}'.format))
    plt.grid(True, linestyle='--', alpha=0.6)

    plt.tight_layout()
    plt.show()

    print("\n--- Model Validation (V-11 C.1) ---")
    print(f"  Avg. collateral change after 1 successful repayment: {avg_collateral_change*100:.2f}%")
    print(f"  (Compare this to the '-3.5%' claim in your paper.)")

# ==============================================================================
# --- SIMULATION 2: Esteva et al. Data (Validates rkAMM Engine) ---
# ==============================================================================

def run_esteva_data_validation(df_invoices, collateral_level):
    """
    Applies our Pricing and Enforcement Agents to the loaded Esteva data.
    """
    print(f"  Running simulation with C = {collateral_level*100:.0f}%...")

    # Initialize the agents needed for this simulation
    pricing_agent = PricingAgent()
    blockchain_agent = BlockchainEnforcementAgent()

    results = []
    total_loans = len(df_invoices)

    for i, row in df_invoices.iterrows():
        # 1. Get p_d from the Esteva dataset
        p_d = row['p_d']

        # 2. Our Pricing Agent calculates the premium
        premium, loss_given_default = pricing_agent.calculate_premium(
            p_d, collateral_level
        )

        # 3. Our Enforcement Agent simulates the outcome
        outcome = blockchain_agent.execute_loan(p_d)

        # 4. Calculate profit/loss
        if outcome == "Repaid":
            net_profit_loss = premium
        else: # Default
            net_profit_loss = premium - loss_given_default

        results.append({
            "loan_id": i,
            "p_d": p_d,
            "collateral_level": collateral_level,
            "premium": premium,
            "loss_given_default": loss_given_default,
            "outcome": outcome,
            "net_profit_loss": net_profit_loss
        })

        # Print progress
        if (i + 1) % 1000 == 0:
            sys.stdout.write(f"\r    ...processed {i+1}/{total_loans} loans.")
            sys.stdout.flush()

    print("\n  ...Run complete.")
    return pd.DataFrame(results)

# ==============================================================================
# --- MAIN EXECUTION CONTROLLER ---
# ==============================================================================

if __name__ == "__main__":

    # --- CHOOSE WHICH SIMULATION TO RUN ---
    # 'DYNAMIC_MAS': Runs the AI agent simulation to validate V-11 Appendix claims.
    # 'ESTEVA_VALIDATION': Runs the pricing engine on the Esteva et al. dataset.

    SIMULATION_TO_RUN = 'DYNAMIC_MAS'
    # SIMULATION_TO_RUN = 'ESTEVA_VALIDATION'

    # --------------------------------------------------------------------------

    if SIMULATION_TO_RUN == 'DYNAMIC_MAS':
        # Run the simulation to validate your V-11 paper's claims
        df_dynamic_results = run_dynamic_mas_simulation(
            n_borrowers=500,
            loans_per_borrower=20
        )

        # Plot the results
        plot_dynamic_results(df_dynamic_results)

        # Save the 10,000 data points if needed
        output_file = "dynamic_mas_10000_loans.csv"
        df_dynamic_results.to_csv(output_file, index=False)
        print(f"\nSaved dynamic simulation data to '{output_file}'")

    # --------------------------------------------------------------------------

    elif SIMULATION_TO_RUN == 'ESTEVA_VALIDATION':
        # Run the simulation to validate against the Esteva et al. (2023) data
        DATA_URL = "https://raw.githubusercontent.com/ballesterosbr/rkAMM/master/notebooks/simulated/l_invoices_10k_0.6_0.3_0.01.csv"

        print(f"--- Running Esteva et al. (2023) Data Validation ---")
        print(f"Downloading dataset from:\n{DATA_URL}\n")

        try:
            df_invoices = pd.read_csv(DATA_URL)
            print(f"Successfully loaded {len(df_invoices)} invoices.")
            print("Dataset 'p_d' distribution (as per Esteva et al.):")
            print(df_invoices['p_d'].value_counts(normalize=True))
            print("\n" + "="*40 + "\n")

        except Exception as e:
            print(f"Error: Could not download or read the file. {e}")
            print("Please check the URL or your internet connection.")
            sys.exit(1)

        # Run for 0% and 60% collateral scenarios
        scenarios_to_run = [0.0, 0.6]
        all_esteva_results = []

        for c_level in scenarios_to_run:
            df_result = run_esteva_data_validation(df_invoices, c_level)
            all_esteva_results.append(df_result)

            print(f"\n--- Financial Summary (C = {c_level*100:.0f}%) ---")
            total_premiums = df_result['premium'].sum()
            total_defaults = len(df_result[df_result['outcome'] == 'Default'])
            total_losses_paid = df_result[df_result['outcome'] == 'Default']['